In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

In [0]:
pin_file_location = "/mnt/pinterest-data/topics/12a3410ba3cf.pin/partition=0//*.json"
file_type = "json"
infer_schema = "true"
df_pin = spark.read.format(file_type) \
                .option("inferSchema", infer_schema) \
                .load(pin_file_location)
geo_file_location = "/mnt/pinterest-data/topics/12a3410ba3cf.geo/partition=0//*.json"
df_geo = spark.read.format(file_type) \
                 .option("inferSchema", infer_schema) \
                 .load(geo_file_location)
user_file_location = "/mnt/pinterest-data/topics/12a3410ba3cf.user/partition=0//*.json"
df_user = spark.read.format(file_type) \
                  .option("inferSchema", infer_schema) \
                  .load(user_file_location)

In [0]:
df_pin_cleaned = df_pin.withColumn("category", when(df_pin["category"].isNull(), None).otherwise(df_pin["category"]))
df_pin_cleaned = df_pin.withColumn("description", when(df_pin["description"].isNull(), None).otherwise(df_pin["description"]))
df_pin_cleaned = df_pin_cleaned.withColumn(
    "follower_count",
    when(col("follower_count").endswith("k"), 
         regexp_replace(col("follower_count"), "k", "").cast("int") * 1000)
    .when(col("follower_count").endswith("M"), 
          regexp_replace(col("follower_count"), "M", "").cast("int") * 1000000)
    .otherwise(col("follower_count").cast("int"))
)
df_pin_cleaned = df_pin_cleaned.withColumn(
    "follower_count",
    when(
        df_pin_cleaned["follower_count"].cast("int").isNotNull(),
        df_pin_cleaned["follower_count"].cast("int")
    ).otherwise(None)
)
df_pin_cleaned.withColumn("downloaded",col("downloaded").cast("int"))
df_pin_cleaned = df_pin_cleaned.withColumn("save_location",regexp_replace(col("save_location"), "^Local save in ", ""))
df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")
desired_column_order = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"]
df_pin_cleaned = df_pin_cleaned.select(desired_column_order)
df_pin_cleaned.withColumn("ind",col("ind").cast("int"))


Out[147]: DataFrame[ind: int, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]

In [0]:
df_geo_cleaned = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))
df_geo_cleaned = df_geo_cleaned.drop("latitude", "longitude")
df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", to_timestamp(col("timestamp")))
df_geo_cleaned = df_geo_cleaned.select("ind", "country", "coordinates", "timestamp")

In [0]:
df_user_cleaned = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))
df_user_cleaned = df_user_cleaned.drop("first_name", "last_name")
df_user_cleaned = df_user_cleaned.withColumn("date_joined", to_timestamp(col("date_joined"), "yyyy-MM-dd'T'HH:mm:ss"))
df_user_cleaned = df_user_cleaned.select("ind", "user_name", "age", "date_joined")

In [0]:
# Find the most popular Pinterest category people post to based on their country
df_pin_cleaned.createOrReplaceGlobalTempView("pin_df")
df_geo_cleaned.createOrReplaceGlobalTempView("geo_df")
df_user_cleaned.createOrReplaceGlobalTempView("user_df")

results_df = spark.sql("""
WITH ranked_categories AS (
    SELECT
        g.country,
        p.category,
        COUNT(*) AS category_count,
        ROW_NUMBER() OVER (PARTITION BY g.country ORDER BY COUNT(*) DESC) AS rank
    FROM
        global_temp.geo_df g
    JOIN
        global_temp.pin_df p
    ON
        g.ind = p.ind
    GROUP BY
        g.country,
        p.category
)
SELECT
    country,
    category,
    category_count
FROM
    ranked_categories
WHERE
    rank = 1
""")

display(results_df)

country,category,category_count
Afghanistan,education,24
Albania,art,38
Algeria,quotes,50
American Samoa,tattoos,16
Andorra,tattoos,11
Angola,education,4
Anguilla,diy-and-crafts,9
Antarctica (the territory South of 60 deg S),tattoos,10
Antigua and Barbuda,art,7
Argentina,tattoos,19


In [0]:
# Find how many posts each category had between 2018 and 2022
df_pin_cleaned.createOrReplaceGlobalTempView("pin_df")
df_geo_cleaned.createOrReplaceGlobalTempView("geo_df")

year_result = spark.sql("""
SELECT
    YEAR(g.timestamp) AS post_year,
    p.category,
    COUNT(*) AS category_count
FROM
    global_temp.geo_df g
JOIN
    global_temp.pin_df p
ON
    g.ind = p.ind
WHERE
    YEAR(g.timestamp) BETWEEN 2018 AND 2022
GROUP BY
    post_year,
    p.category
ORDER BY
    post_year,
    p.category
""")

display(year_result)

post_year,category,category_count
2018,art,43
2018,beauty,30
2018,christmas,47
2018,diy-and-crafts,50
2018,education,43
2018,event-planning,36
2018,finance,32
2018,home-decor,29
2018,mens-fashion,37
2018,quotes,46


In [0]:
# For each country find the user with the most followers
df_pin_cleaned.createOrReplaceGlobalTempView("pin_df")
df_geo_cleaned.createOrReplaceGlobalTempView("geo_df")

follower_result = spark.sql("""
WITH RankedUsers AS (
    SELECT
        g.country,
        p.poster_name,
        p.follower_count,
        ROW_NUMBER() OVER (PARTITION BY g.country ORDER BY p.follower_count DESC) AS rank
    FROM
        global_temp.geo_df g
    JOIN
        global_temp.pin_df p
    ON
        g.ind = p.ind
)

SELECT
    country,
    poster_name,
    follower_count
FROM
    RankedUsers
WHERE
    rank = 1
""")

display(follower_result)

country,poster_name,follower_count
Afghanistan,9GAG,3000000
Albania,The Minds Journal,5000000
Algeria,Apartment Therapy,5000000
American Samoa,Mamas Uncut,8000000
Andorra,Teachers Pay Teachers,1000000
Angola,Tastemade,8000000
Anguilla,We Heart It,15000000
Antarctica (the territory South of 60 deg S),Refinery29,1000000
Antigua and Barbuda,Country Living Magazine,1000000
Argentina,Cheezburger,2000000


country,follower_count
Afghanistan,3000000
Albania,5000000
Algeria,5000000
American Samoa,8000000
Andorra,1000000
Angola,8000000
Anguilla,15000000
Antarctica (the territory South of 60 deg S),1000000
Antigua and Barbuda,1000000
Argentina,2000000


In [0]:
# Find the most popular category for each age group
df_pin_cleaned.createOrReplaceGlobalTempView("pin_df")
df_user_cleaned.createOrReplaceGlobalTempView("user_df")

age_df = spark.sql("""
WITH AgeGroups AS (
    SELECT
        u.age,
        p.category,
        CASE
            WHEN u.age BETWEEN 18 AND 24 THEN '18-24'
            WHEN u.age BETWEEN 25 AND 35 THEN '25-35'
            WHEN u.age BETWEEN 36 AND 50 THEN '36-50'
            ELSE '+50'
        END AS age_group
    FROM
        global_temp.user_df u
    JOIN
        global_temp.pin_df p
    ON
        u.ind = p.ind
),

RankedCategories AS (
    SELECT
        ag.age_group,
        ag.category,
        COUNT(*) AS category_count,
        ROW_NUMBER() OVER (PARTITION BY ag.age_group ORDER BY COUNT(*) DESC) AS rank
    FROM
        AgeGroups ag
    GROUP BY
        ag.age_group,
        ag.category
)

SELECT
    rc.age_group,
    rc.category AS top_category,
    rc.category_count AS top_category_count
FROM
    RankedCategories rc
WHERE
    rc.rank = 1
""")

display(age_df)

age_group,top_category,top_category_count
+50,vehicles,27
18-24,art,133
25-35,education,70
36-50,travel,56


In [0]:
#Find the median follower count for different age group
df_pin_cleaned.createOrReplaceGlobalTempView("pin_df")
df_user_cleaned.createOrReplaceGlobalTempView("user_df")

med_follower_count_df = spark.sql("""
WITH AgeGroups AS (
    SELECT
        u.age,
        p.follower_count,
        CASE
            WHEN u.age BETWEEN 18 AND 24 THEN '18-24'
            WHEN u.age BETWEEN 25 AND 35 THEN '25-35'
            WHEN u.age BETWEEN 36 AND 50 THEN '36-50'
            ELSE '+50'
        END AS age_group
    FROM
        global_temp.user_df u
    JOIN
        global_temp.pin_df p
    ON
        u.ind = p.ind
)

SELECT
    age_group,
    percentile(follower_count, 0.5) AS median_follower_count
FROM
    AgeGroups
GROUP BY
    age_group
""")

display(med_follower_count_df)

age_group,median_follower_count
36-50,7000.0
+50,877.0
18-24,123000.0
25-35,22000.0


In [0]:
# Find how many users have joined between 2015 and 2020
df_user_cleaned.createOrReplaceGlobalTempView("user_df")

user_joined_df = spark.sql("""
SELECT
    YEAR(date_joined) AS post_year,
    COUNT(*) AS number_users_joined
FROM
    global_temp.user_df
WHERE
    YEAR(date_joined) BETWEEN 2015 AND 2020
GROUP BY
    post_year
ORDER BY
    post_year
""")

display(user_joined_df)

post_year,number_users_joined
2015,804
2016,929
2017,327


In [0]:
# Find the median follower count of users have joined between 2015 and 2020
df_user_cleaned.createOrReplaceGlobalTempView("user_df")

med_users_df = spark.sql("""
SELECT
    YEAR(u.date_joined) AS post_year,
    percentile_approx(p.follower_count, 0.5) AS median_follower_count
FROM
    global_temp.user_df u
JOIN
    global_temp.df_pin p
ON
    u.ind = p.ind
WHERE
    YEAR(u.date_joined) BETWEEN 2015 AND 2020
GROUP BY
    post_year
ORDER BY
    post_year
""")

display(med_users_df)

post_year,median_follower_count
2015,166000
2016,17000
2017,3000


In [0]:
# Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of
df_user_cleaned.createOrReplaceGlobalTempView("user_df")

med_users_age_group_df = spark.sql("""
WITH UserAgeGroups AS (
    SELECT
        CASE
            WHEN age >= 18 AND age <= 24 THEN '18-24'
            WHEN age >= 25 AND age <= 35 THEN '25-35'
            WHEN age >= 36 AND age <= 50 THEN '36-50'
            ELSE '+50'
        END AS age_group,
        YEAR(u.date_joined) AS post_year,
        p.follower_count AS follower_count
    FROM
        global_temp.user_df u
    JOIN
        global_temp.pin_df p
    ON
        u.ind = p.ind
    WHERE
        YEAR(u.date_joined) BETWEEN 2015 AND 2020
)
SELECT
    age_group,
    post_year,
    percentile_approx(follower_count, 0.5) AS median_follower_count
FROM
    UserAgeGroups
GROUP BY
    age_group,
    post_year
ORDER BY
    age_group,
    post_year
""")

display(med_users_age_group_df)

age_group,post_year,median_follower_count
+50,2015,14000
+50,2016,504
+50,2017,1000
18-24,2015,228000
18-24,2016,41000
18-24,2017,9000
25-35,2015,42000
25-35,2016,21000
25-35,2017,6000
36-50,2015,13000
